# Spam Classification

<td>
    <a href="#" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Downloading data